# Query-based retrieval

Information retrieval for AI agents is rarely a simple matter of finding documents that match keywords. Production knowledge bases contain thousands or millions of documents with varying levels of relevance, authority, and freshness. A naive search might return technically matching documents that are outdated, from untrusted sources, or semantically off-target. The challenge is not just finding potential matches, but systematically filtering through multiple dimensions to identify the small subset of documents that truly deserve a place in the agent's context.

Query-based retrieval addresses this challenge through multi-stage filtering pipelines that progressively narrow the candidate set using different criteria. Rather than relying on a single similarity score, we combine semantic filters, metadata constraints, recency requirements, source authority ratings, and hybrid search strategies to select documents that are both semantically relevant and contextually appropriate. Each filter removes documents that fail specific quality or relevance criteria, ensuring that only the highest-quality, most pertinent information reaches the agent.

In this notebook, we explore how to build sophisticated query-based retrieval systems as a select strategy for context engineering. We will examine how to implement semantic similarity filters that capture meaning beyond keywords, how to apply metadata filters for attributes like document type or category, how to enforce recency requirements for time-sensitive domains, how to score and prioritize based on source authority, and how to combine multiple retrieval strategies through hybrid search.

In [1]:
import os
from datetime import datetime, timedelta
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass

We begin by initializing the language model and embedding model that will power our retrieval pipeline. Consistent models ensure reproducible similarity measurements across all filtering stages.

In [2]:
# Initialize the language model for generating responses
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY", "").strip(),
    temperature=0  # Set to 0 for more deterministic outputs
)

# Initialize embedding model for semantic search
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=os.getenv("OPENAI_API_KEY", "").strip())

print("Models initialized successfully!")

Models initialized successfully!


## Building a rich knowledge base with metadata

Effective multi-stage filtering requires rich document metadata that captures various dimensions of relevance and quality. Unlike simple text collections, production knowledge bases should include attributes like document type, publication date, source authority, category tags, and version information. This metadata enables sophisticated filtering that goes beyond semantic similarity to consider recency, trustworthiness and topical alignment.

We will create a realistic knowledge base for a technology company's customer support system, with documents spanning product documentation, troubleshooting guides, policy statements and announcements. Each document includes comprehensive metadata that our filtering stages will leverage to make intelligent selection decisions.

In [3]:
@dataclass
class DocumentMetadata:
    """Rich metadata for knowledge base documents."""
    doc_type: str  # Type: 'documentation', 'policy', 'troubleshooting', 'announcement'
    category: str  # Product category or topic area
    source_authority: int  # Authority score 1-5 (5 = official, 1 = community)
    publish_date: datetime  # When document was created/updated
    version: str  # Document version
    author: str  # Document author or team

# Create comprehensive knowledge base with rich metadata
def create_knowledge_base() -> List[Document]:
    """Create knowledge base documents with comprehensive metadata.
    
    Returns:
        List of Document objects with content and metadata
    """
    now = datetime.now()
    
    documents = [
        # Recent official documentation
        Document(
            page_content="Laptop Pro X1 overheating: Ensure ventilation is clear. Update thermal management drivers to v2.3.1. Use cooling pad for intensive gaming.",
            metadata={
                "doc_type": "troubleshooting",
                "category": "laptops",
                "source_authority": 5,
                "publish_date": (now - timedelta(days=10)).isoformat(),
                "version": "2.3",
                "author": "Technical Support Team"
            }
        ),
        
        # Older documentation (outdated)
        Document(
            page_content="Laptop overheating issues: Clean fans and reapply thermal paste. Update to driver v1.8 for better cooling.",
            metadata={
                "doc_type": "troubleshooting",
                "category": "laptops",
                "source_authority": 5,
                "publish_date": (now - timedelta(days=400)).isoformat(),
                "version": "1.8",
                "author": "Technical Support Team"
            }
        ),
        
        # Community contribution (lower authority)
        Document(
            page_content="My laptop runs hot too. I just put it in the fridge for a few minutes when it overheats. Works great!",
            metadata={
                "doc_type": "troubleshooting",
                "category": "laptops",
                "source_authority": 1,  # Community post, not official
                "publish_date": (now - timedelta(days=5)).isoformat(),
                "version": "1.0",
                "author": "CommunityUser123"
            }
        ),
        
        # Official policy documents
        Document(
            page_content="Return Policy 2024: Products may be returned within 30 days of purchase with original packaging. Refunds processed within 5-7 business days.",
            metadata={
                "doc_type": "policy",
                "category": "returns",
                "source_authority": 5,
                "publish_date": (now - timedelta(days=60)).isoformat(),
                "version": "2024.1",
                "author": "Legal Team"
            }
        ),
        
        # Old policy (superseded)
        Document(
            page_content="Return Policy 2023: Products may be returned within 14 days. Refunds take 10-14 business days.",
            metadata={
                "doc_type": "policy",
                "category": "returns",
                "source_authority": 5,
                "publish_date": (now - timedelta(days=450)).isoformat(),
                "version": "2023.1",
                "author": "Legal Team"
            }
        ),
        
        # Product documentation
        Document(
            page_content="Laptop Pro X1 specifications: 16GB RAM, 512GB SSD, Intel i7 processor, 15.6 inch display. Battery life up to 10 hours.",
            metadata={
                "doc_type": "documentation",
                "category": "laptops",
                "source_authority": 5,
                "publish_date": (now - timedelta(days=30)).isoformat(),
                "version": "1.0",
                "author": "Product Team"
            }
        ),
        
        # Shipping information
        Document(
            page_content="Shipping: Standard delivery 5-7 business days, Express 2-3 days. Free shipping on orders over $50.",
            metadata={
                "doc_type": "policy",
                "category": "shipping",
                "source_authority": 4,
                "publish_date": (now - timedelta(days=90)).isoformat(),
                "version": "1.5",
                "author": "Operations Team"
            }
        ),
        
        # Recent announcement
        Document(
            page_content="NEW: Extended warranty now available for all laptop models. 2-year coverage for $199.",
            metadata={
                "doc_type": "announcement",
                "category": "laptops",
                "source_authority": 5,
                "publish_date": (now - timedelta(days=3)).isoformat(),
                "version": "1.0",
                "author": "Marketing Team"
            }
        ),
        
        # Tablet troubleshooting (different category)
        Document(
            page_content="Tablet Mini screen issues: Calibrate touchscreen in settings. Reset display settings if unresponsive.",
            metadata={
                "doc_type": "troubleshooting",
                "category": "tablets",
                "source_authority": 5,
                "publish_date": (now - timedelta(days=20)).isoformat(),
                "version": "1.2",
                "author": "Technical Support Team"
            }
        ),
        
        # Battery troubleshooting
        Document(
            page_content="Laptop battery draining fast: Check for background processes. Reduce screen brightness. Update power management drivers.",
            metadata={
                "doc_type": "troubleshooting",
                "category": "laptops",
                "source_authority": 5,
                "publish_date": (now - timedelta(days=15)).isoformat(),
                "version": "2.1",
                "author": "Technical Support Team"
            }
        ),
    ]
    
    return documents

# Create knowledge base
kb_documents = create_knowledge_base()

print(f"Created knowledge base with {len(kb_documents)} documents")
print(f"\nSample document metadata:")
print(f"Content: {kb_documents[0].page_content}")
print(f"Metadata: {kb_documents[0].metadata}")

Created knowledge base with 10 documents

Sample document metadata:
Content: Laptop Pro X1 overheating: Ensure ventilation is clear. Update thermal management drivers to v2.3.1. Use cooling pad for intensive gaming.
Metadata: {'doc_type': 'troubleshooting', 'category': 'laptops', 'source_authority': 5, 'publish_date': '2025-11-26T15:55:39.674713', 'version': '2.3', 'author': 'Technical Support Team'}


Our knowledge base now contains documents with diverse characteristics:
1. Creates documents with rich metadata including type, category, authority scores, publication dates, versions and authors.
2. Includes intentional variety to demonstrate filtering effectiveness—outdated documents, low-authority sources, different categories and varying recency.
3. Uses realistic content for a tech support scenario with troubleshooting guides, policies, documentation and announcements.
4. Provides the foundation for demonstrating how multi-stage filtering progressively narrows results to the most relevant, trustworthy, current information.

This diverse collection makes an ideal testbed for advanced retrieval strategies.

## Stage 1 - Semantic similarity filtering

The first filtering stage uses semantic similarity to identify documents that are conceptually related to the query. This casts a wide net, retrieving all documents that might be relevant based on meaning rather than exact keyword matches. We typically retrieve more candidates than needed at this stage, knowing that subsequent filters will narrow the results based on other criteria.

Semantic filtering forms the foundation of the pipeline by ensuring we start with documents that are at least topically related. Without this initial semantic filter, metadata-based filters would have no way to distinguish between documents that are completely irrelevant versus those that are relevant but perhaps outdated or from lower-authority sources.

In [4]:
# Create vector store from knowledge base for semantic search
vector_store = FAISS.from_documents(kb_documents, embeddings)

def semantic_filter(query: str, 
                   vector_store: FAISS,
                   similarity_threshold: float = 0.5,
                   top_k: int = 10) -> List[Tuple[Document, float]]:
    """Stage 1: Filter by semantic similarity to query.
    
    Args:
        query: User's search query
        vector_store: Vector store containing knowledge base
        similarity_threshold: Minimum similarity score (0-1)
        top_k: Maximum candidates to retrieve
        
    Returns:
        List of (document, similarity_score) tuples above threshold
    """
    # Retrieve semantically similar documents with scores
    results = vector_store.similarity_search_with_score(query, k=top_k)
    
    # Return all results (threshold filtering can happen in later stages)
    return results

# Test semantic filtering
query = "My laptop is getting too hot when I use it"

print(f"Query: '{query}'")
print(f"\nStage 1: Semantic Similarity Filter")
print("=" * 70)

semantic_results = semantic_filter(query, vector_store, top_k=5)

print(f"Retrieved {len(semantic_results)} semantically similar documents:\n")

for i, (doc, score) in enumerate(semantic_results, 1):
    print(f"{i}. [Similarity: {score:.4f}]")
    print(f"   Content: {doc.page_content[:80]}...")
    print(f"   Type: {doc.metadata['doc_type']} | Category: {doc.metadata['category']}")
    print(f"   Authority: {doc.metadata['source_authority']}/5 | Published: {doc.metadata['publish_date'][:10]}")
    print()

Query: 'My laptop is getting too hot when I use it'

Stage 1: Semantic Similarity Filter
Retrieved 5 semantically similar documents:

1. [Similarity: 0.2439]
   Content: My laptop runs hot too. I just put it in the fridge for a few minutes when it ov...
   Type: troubleshooting | Category: laptops
   Authority: 1/5 | Published: 2025-12-01

2. [Similarity: 0.2789]
   Content: Laptop overheating issues: Clean fans and reapply thermal paste. Update to drive...
   Type: troubleshooting | Category: laptops
   Authority: 5/5 | Published: 2024-11-01

3. [Similarity: 0.3006]
   Content: Laptop Pro X1 overheating: Ensure ventilation is clear. Update thermal managemen...
   Type: troubleshooting | Category: laptops
   Authority: 5/5 | Published: 2025-11-26

4. [Similarity: 0.3461]
   Content: Laptop battery draining fast: Check for background processes. Reduce screen brig...
   Type: troubleshooting | Category: laptops
   Authority: 5/5 | Published: 2025-11-21

5. [Similarity: 0.4145]
   Content

The semantic filter successfully identifies relevant documents:
1. Implements vector similarity search to retrieve documents semantically related to the query about laptop overheating.
2. Returns top candidates with similarity scores, capturing both highly relevant troubleshooting guides and tangentially related content.
3. Demonstrates that semantic similarity alone may surface outdated documents, low-authority sources and varying quality content.
4. Sets the stage for additional filtering layers that will refine this candidate set based on metadata criteria.

Semantic filtering ensures we start with topically relevant documents before applying quality filters.

## Stage 2 - Metadata filtering

The second filtering stage applies metadata constraints to narrow the candidate set based on document attributes. This might include filtering by document type to include only troubleshooting guides and exclude announcements, filtering by category to ensure topical alignment, or filtering by other custom attributes that indicate relevance to the current context.

Metadata filters are precise and efficient because they operate on structured attributes rather than semantic similarity scores. They allow us to enforce hard requirements - for example, when handling a refund inquiry, we might require documents of type policy, or when troubleshooting a laptop issue, we might exclude documents categorized under tablets. These filters dramatically improve precision by removing semantically similar but contextually inappropriate documents.

In [5]:
def metadata_filter(candidates: List[Tuple[Document, float]],
                   doc_types: Optional[List[str]] = None,
                   categories: Optional[List[str]] = None,
                   min_authority: Optional[int] = None) -> List[Tuple[Document, float]]:
    """Stage 2: Filter by document metadata attributes.
    
    Args:
        candidates: Results from semantic filtering
        doc_types: Allowed document types (None = all types)
        categories: Allowed categories (None = all categories)
        min_authority: Minimum source authority score (None = no filter)
        
    Returns:
        Filtered list of (document, score) tuples
    """
    filtered = []
    
    for doc, score in candidates:
        # Check document type filter
        if doc_types and doc.metadata.get('doc_type') not in doc_types:
            continue
        
        # Check category filter
        if categories and doc.metadata.get('category') not in categories:
            continue
        
        # Check authority filter
        if min_authority and doc.metadata.get('source_authority', 0) < min_authority:
            continue
        
        # Document passes all filters
        filtered.append((doc, score))
    
    return filtered

# Apply metadata filters to semantic results
print(f"Stage 2: Metadata Filter")
print("=" * 70)

# Filter for troubleshooting docs in laptops category with high authority
metadata_results = metadata_filter(
    semantic_results,
    doc_types=['troubleshooting'],  # Only troubleshooting guides
    categories=['laptops'],  # Only laptop-related
    min_authority=4  # Minimum authority of 4/5
)

print(f"Applied filters:")
print(f"  - Document types: ['troubleshooting']")
print(f"  - Categories: ['laptops']")
print(f"  - Minimum authority: 4/5")
print(f"\nFiltered from {len(semantic_results)} to {len(metadata_results)} documents:\n")

for i, (doc, score) in enumerate(metadata_results, 1):
    print(f"{i}. [Similarity: {score:.4f}]")
    print(f"   Content: {doc.page_content[:80]}...")
    print(f"   Type: {doc.metadata['doc_type']} | Category: {doc.metadata['category']}")
    print(f"   Authority: {doc.metadata['source_authority']}/5")
    print()

Stage 2: Metadata Filter
Applied filters:
  - Document types: ['troubleshooting']
  - Categories: ['laptops']
  - Minimum authority: 4/5

Filtered from 5 to 3 documents:

1. [Similarity: 0.2789]
   Content: Laptop overheating issues: Clean fans and reapply thermal paste. Update to drive...
   Type: troubleshooting | Category: laptops
   Authority: 5/5

2. [Similarity: 0.3006]
   Content: Laptop Pro X1 overheating: Ensure ventilation is clear. Update thermal managemen...
   Type: troubleshooting | Category: laptops
   Authority: 5/5

3. [Similarity: 0.3461]
   Content: Laptop battery draining fast: Check for background processes. Reduce screen brig...
   Type: troubleshooting | Category: laptops
   Authority: 5/5



Metadata filtering significantly refines the candidate set:
1. Implements flexible metadata filtering that can constrain by document type, category and authority level based on use case requirements.
2. Applies filters to remove documents that are semantically similar but contextually inappropriate (e.g., announcements when troubleshooting is needed).
3. Enforces minimum authority requirements to exclude low-quality community contributions when official guidance is needed.
4. Demonstrates how metadata filters work in conjunction with semantic similarity to balance breadth and precision.

The combination of semantic and metadata filtering produces focused, high-quality results.

## Stage 3 - Recency filtering

For many domains, information freshness is critical. Technical documentation becomes outdated as products evolve, policies change over time, and troubleshooting procedures improve with new discoveries. The third filtering stage applies recency requirements to ensure we surface current information and suppress superseded content.

Recency filtering can be implemented in multiple ways. Hard cutoffs exclude any document older than a specific age, useful when we know information becomes invalid after a certain period. Recency boosting adjusts similarity scores based on document age, allowing recent documents to rank higher without completely excluding older content. The right approach depends on whether outdated information is merely less useful or actively harmful to include.

In [6]:
def recency_filter(candidates: List[Tuple[Document, float]],
                  max_age_days: Optional[int] = None,
                  boost_recent: bool = False,
                  boost_factor: float = 0.1) -> List[Tuple[Document, float]]:
    """Stage 3: Filter or boost based on document recency.
    
    Args:
        candidates: Results from previous filtering stages
        max_age_days: Maximum document age in days (None = no hard cutoff)
        boost_recent: Whether to boost scores of recent documents
        boost_factor: Score boost per 30 days of recency (if boosting enabled)
        
    Returns:
        Filtered/boosted list of (document, score) tuples
    """
    now = datetime.now()
    results = []
    
    for doc, score in candidates:
        # Parse publication date
        pub_date_str = doc.metadata.get('publish_date', '')
        try:
            pub_date = datetime.fromisoformat(pub_date_str)
        except:
            # If date parsing fails, exclude document
            continue
        
        # Calculate age in days
        age_days = (now - pub_date).days
        
        # Apply hard cutoff if specified
        if max_age_days and age_days > max_age_days:
            continue
        
        # Apply recency boost if enabled
        adjusted_score = score
        if boost_recent:
            # Boost score based on recency (more recent = higher boost)
            recency_boost = (max_age_days - age_days) / 30 * boost_factor if max_age_days else 0
            adjusted_score = score + recency_boost
        
        results.append((doc, adjusted_score))
    
    # Re-sort by adjusted scores
    results.sort(key=lambda x: x[1], reverse=True)
    
    return results

print(f"Stage 3: Recency Filter")
print("=" * 70)

# Apply recency filter with 180-day maximum age
recency_results = recency_filter(
    metadata_results,
    max_age_days=180,  # Only documents from last 6 months
    boost_recent=True,  # Boost recent documents
    boost_factor=0.05
)

print(f"Applied filters:")
print(f"  - Maximum age: 180 days")
print(f"  - Recency boosting: enabled")
print(f"\nFiltered from {len(metadata_results)} to {len(recency_results)} documents:\n")

for i, (doc, score) in enumerate(recency_results, 1):
    pub_date = datetime.fromisoformat(doc.metadata['publish_date'])
    age_days = (datetime.now() - pub_date).days
    
    print(f"{i}. [Adjusted Score: {score:.4f}]")
    print(f"   Content: {doc.page_content[:80]}...")
    print(f"   Published: {doc.metadata['publish_date'][:10]} ({age_days} days ago)")
    print(f"   Version: {doc.metadata['version']}")
    print()

Stage 3: Recency Filter
Applied filters:
  - Maximum age: 180 days
  - Recency boosting: enabled

Filtered from 3 to 2 documents:

1. [Adjusted Score: 0.6211]
   Content: Laptop battery draining fast: Check for background processes. Reduce screen brig...
   Published: 2025-11-21 (15 days ago)
   Version: 2.1

2. [Adjusted Score: 0.5840]
   Content: Laptop Pro X1 overheating: Ensure ventilation is clear. Update thermal managemen...
   Published: 2025-11-26 (10 days ago)
   Version: 2.3



Recency filtering ensures we surface current, relevant information:
1. Implements both hard cutoff filtering (excluding documents older than specified age) and recency boosting (increasing scores for newer documents).
2. Calculates document age from publication date metadata and applies age-based filters and adjustments.
3. Removes outdated content that may contain superseded information (older driver versions, previous policies, etc.).
4. Re-ranks results to prioritize recent documents, ensuring users see the most current guidance first.

For technical support and policy domains, recency filtering is essential for accuracy.

## Stage 4 - Source authority scoring

Not all documents are equally trustworthy. Official documentation from product teams should be weighted more heavily than community forum posts, legal policy documents should take precedence over blog summaries, and verified expert content should outrank unverified contributions. The fourth filtering stage uses source authority scores to prioritize high-quality, trustworthy information.

Authority scoring can be implemented as hard filtering that excludes low-authority sources entirely, or as score boosting that increases the ranking of high-authority documents while still allowing lower-authority content to appear if it is highly semantically relevant. The approach should match our quality requirements and the diversity of our source base.

In [7]:
def authority_scoring(candidates: List[Tuple[Document, float]],
                     boost_factor: float = 0.1) -> List[Tuple[Document, float]]:
    """Stage 4: Boost scores based on source authority.
    
    Args:
        candidates: Results from previous filtering stages
        boost_factor: Score boost per authority point
        
    Returns:
        Re-ranked list of (document, score) tuples
    """
    results = []
    
    for doc, score in candidates:
        # Get authority score (1-5)
        authority = doc.metadata.get('source_authority', 1)
        
        # Boost score based on authority
        # Authority 5 gets highest boost, authority 1 gets minimal boost
        authority_boost = (authority - 1) * boost_factor
        adjusted_score = score + authority_boost
        
        results.append((doc, adjusted_score))
    
    # Re-sort by adjusted scores
    results.sort(key=lambda x: x[1], reverse=True)
    
    return results

print(f"Stage 4: Source Authority Scoring")
print("=" * 70)

# Apply authority-based boosting
authority_results = authority_scoring(
    recency_results,
    boost_factor=0.1  # Boost score by 0.1 per authority point
)

print(f"Applied authority-based score boosting")
print(f"  - Boost factor: 0.1 per authority point")
print(f"  - Official sources (5/5) get maximum boost")
print(f"\nRe-ranked {len(authority_results)} documents by authority:\n")

for i, (doc, score) in enumerate(authority_results, 1):
    authority = doc.metadata['source_authority']
    
    print(f"{i}. [Final Score: {score:.4f}]")
    print(f"   Content: {doc.page_content[:80]}...")
    print(f"   Authority: {authority}/5 ({doc.metadata['author']})")
    print(f"   Version: {doc.metadata['version']}")
    print()

Stage 4: Source Authority Scoring
Applied authority-based score boosting
  - Boost factor: 0.1 per authority point
  - Official sources (5/5) get maximum boost

Re-ranked 2 documents by authority:

1. [Final Score: 1.0211]
   Content: Laptop battery draining fast: Check for background processes. Reduce screen brig...
   Authority: 5/5 (Technical Support Team)
   Version: 2.1

2. [Final Score: 0.9840]
   Content: Laptop Pro X1 overheating: Ensure ventilation is clear. Update thermal managemen...
   Authority: 5/5 (Technical Support Team)
   Version: 2.3



Authority scoring ensures trustworthy sources rise to the top:
1. Implements score boosting based on source authority ratings, rewarding official documentation and penalizing low-quality sources.
2. Applies graduated boosting where maximum-authority sources (5/5) receive the highest adjustment while minimum-authority sources (1/5) receive minimal or no boost.
3. Re-ranks the filtered result set so that among similarly relevant documents, higher-authority sources appear first.
4. Demonstrates that authority scoring works in concert with semantic similarity - both relevance and trustworthiness contribute to final ranking.

This ensures agents rely on authoritative information rather than questionable sources.

## Complete multi-stage retrieval pipeline

Now we bring together all filtering stages into a unified pipeline that processes queries through semantic similarity, metadata constraints, recency requirements and authority scoring in sequence. This production-ready system demonstrates how multiple filtering dimensions combine to produce highly relevant, trustworthy, current results from large, diverse knowledge bases.

The multi-stage approach is powerful because each filter addresses a different dimension of quality and relevance. Semantic similarity ensures topical relevance, metadata filters enforce contextual appropriateness, recency filters maintain currency, and authority scoring prioritizes trustworthiness. Together, they create a robust selection mechanism that consistently surfaces the best available information.

In [8]:
class MultiStageRetriever:
    """Production multi-stage retrieval pipeline."""
    
    def __init__(self, vector_store: FAISS):
        """Initialize retriever with vector store.
        
        Args:
            vector_store: FAISS vector store containing knowledge base
        """
        self.vector_store = vector_store
    
    def retrieve(self,
                query: str,
                doc_types: Optional[List[str]] = None,
                categories: Optional[List[str]] = None,
                min_authority: int = 3,
                max_age_days: int = 365,
                top_k: int = 3) -> Dict:
        """Execute multi-stage retrieval pipeline.
        
        Args:
            query: User's search query
            doc_types: Allowed document types
            categories: Allowed categories
            min_authority: Minimum source authority (1-5)
            max_age_days: Maximum document age in days
            top_k: Number of final results to return
            
        Returns:
            Dict containing results and pipeline metadata
        """
        # Stage 1: Semantic similarity
        stage1_results = semantic_filter(query, self.vector_store, top_k=10)
        
        # Stage 2: Metadata filtering
        stage2_results = metadata_filter(
            stage1_results,
            doc_types=doc_types,
            categories=categories,
            min_authority=min_authority
        )
        
        # Stage 3: Recency filtering and boosting
        stage3_results = recency_filter(
            stage2_results,
            max_age_days=max_age_days,
            boost_recent=True,
            boost_factor=0.05
        )
        
        # Stage 4: Authority scoring
        stage4_results = authority_scoring(
            stage3_results,
            boost_factor=0.1
        )
        
        # Take top K results
        final_results = stage4_results[:top_k]
        
        return {
            "query": query,
            "results": final_results,
            "pipeline_stats": {
                "stage1_semantic": len(stage1_results),
                "stage2_metadata": len(stage2_results),
                "stage3_recency": len(stage3_results),
                "stage4_authority": len(stage4_results),
                "final_count": len(final_results)
            }
        }

# Create retriever and test with multiple queries
retriever = MultiStageRetriever(vector_store)

test_queries = [
    {
        "query": "My laptop overheats when gaming",
        "doc_types": ["troubleshooting"],
        "categories": ["laptops"],
        "min_authority": 4,
        "max_age_days": 180
    },
    {
        "query": "What is your return policy?",
        "doc_types": ["policy"],
        "categories": ["returns"],
        "min_authority": 5,
        "max_age_days": 365
    },
]

print("Multi-Stage Retrieval Pipeline - Complete Examples")
print("=" * 70)

for test_case in test_queries:
    result = retriever.retrieve(**test_case)
    
    print(f"\nQuery: '{result['query']}'")
    print(f"\nPipeline progression:")
    stats = result['pipeline_stats']
    print(f"  Stage 1 (Semantic): {stats['stage1_semantic']} candidates")
    print(f"  Stage 2 (Metadata): {stats['stage2_metadata']} candidates")
    print(f"  Stage 3 (Recency):  {stats['stage3_recency']} candidates")
    print(f"  Stage 4 (Authority): {stats['stage4_authority']} candidates")
    print(f"  Final results: {stats['final_count']} documents")
    
    print(f"\nTop {len(result['results'])} results:")
    for i, (doc, score) in enumerate(result['results'], 1):
        print(f"\n  {i}. [Score: {score:.4f}]")
        print(f"     {doc.page_content}")
        print(f"     Type: {doc.metadata['doc_type']} | Authority: {doc.metadata['source_authority']}/5")
        print(f"     Published: {doc.metadata['publish_date'][:10]} | Version: {doc.metadata['version']}")
    
    print(f"\n{'='*70}")

Multi-Stage Retrieval Pipeline - Complete Examples

Query: 'My laptop overheats when gaming'

Pipeline progression:
  Stage 1 (Semantic): 10 candidates
  Stage 2 (Metadata): 3 candidates
  Stage 3 (Recency):  2 candidates
  Stage 4 (Authority): 2 candidates
  Final results: 2 documents

Top 2 results:

  1. [Score: 1.0424]
     Laptop battery draining fast: Check for background processes. Reduce screen brightness. Update power management drivers.
     Type: troubleshooting | Authority: 5/5
     Published: 2025-11-21 | Version: 2.1

  2. [Score: 0.9380]
     Laptop Pro X1 overheating: Ensure ventilation is clear. Update thermal management drivers to v2.3.1. Use cooling pad for intensive gaming.
     Type: troubleshooting | Authority: 5/5
     Published: 2025-11-26 | Version: 2.3


Query: 'What is your return policy?'

Pipeline progression:
  Stage 1 (Semantic): 10 candidates
  Stage 2 (Metadata): 2 candidates
  Stage 3 (Recency):  1 candidates
  Stage 4 (Authority): 1 candidates
  Final

The complete pipeline demonstrates production-quality retrieval:
1. Implements a unified retriever class that orchestrates all four filtering stages in sequence, providing a clean interface for complex retrieval.
2. Tracks pipeline statistics showing how many candidates survive each stage, providing visibility into filtering effectiveness.
3. Tests realistic scenarios with different filtering requirements, demonstrating flexibility to handle diverse query types.
4. Returns only the highest-quality, most relevant documents after progressive refinement through multiple quality dimensions.

This architecture scales to large knowledge bases while maintaining precision and efficiency.